In [1]:
import phys
import phys.light

sim = phys.Simulation({"cl_on": True})
sim.add_objs(phys.light.generate_photons_from_E([phys.light.planck_phot_distribution(phys.light.E_from_wavelength(500000e-9), phys.light.E_from_wavelength(100e-9), 2500, bins=50000) for x in range(100)]))
skip = phys.__CLInput(name="photon_check", type="obj_action", code="if type(obj) != phys.light.PhotonObject:\n \t\t continue")
d0, d1, d2 = tuple([phys.__CLInput(name="d" + str(x), type="obj", obj_attr="v[" + str(x) + "]") for x in range(0, 3)])
rand = phys.__CLInput(name="rand", type="obj_def", obj_def="np.random.random()")
A, n = phys.__CLInput(name="A", type="const", const_value="0.00001"), phys.__CLInput(name="n", type="const", const_value="0.00001")

dbg = phys.__CLInput(name="dbg", type="obj_action", code="print(obj.v)")

pht = phys.__CLInput(name="pht", type="obj_track", obj_track="obj")
res = phys.__CLOutput(name="res", ctype="int")

kernel = """
				int gid = get_global_id(0);
				double norm = sqrt(pow(d0[gid], 2) + pow(d1[gid], 2) + pow(d2[gid], 2));
				double pcoll = A * n * norm;
				if (pcoll >= rand[gid]){
					// Mark for removal.
					res[gid] = 1;
				} else {
					res[gid] = 0;
				}
"""

prog = phys.__CLProgram(sim, "test", kernel)
prog.prep_metadata = [skip, dbg, d0, d1, d2, rand, pht, A, n]
prog.output_metadata = [res]
prog.build_kernel()
out = prog.run()
print(out)

__kernel void test(__global double *d0, __global double *d1, __global double *d2, __global double *rand, double A, double n, __global int *res){
				int gid = get_global_id(0);
				double norm = sqrt(pow(d0[gid], 2) + pow(d1[gid], 2) + pow(d2[gid], 2));
				double pcoll = A * n * norm;
				if (pcoll >= rand[gid]){
					// Mark for removal.
					res[gid] = 1;
				} else {
					res[gid] = 0;
				}
}
self.d0 = []
self.d1 = []
self.d2 = []
self.rand = []
self.pht = []


for obj in self.sim.objects:
	if type(obj) != phys.light.PhotonObject:
 		 continue
	print(obj.v)
	self.d0.append(obj.v[0])
	self.d1.append(obj.v[1])
	self.d2.append(obj.v[2])
	self.rand.append(np.random.random())

self.d0_np = np.array(self.d0, dtype=np.double)
self.d1_np = np.array(self.d1, dtype=np.double)
self.d2_np = np.array(self.d2, dtype=np.double)
self.rand_np = np.array(self.rand, dtype=np.double)


self.d0_dev = cl_array.to_device(self.sim.cl_q, self.d0_np)
self.d1_dev = cl_array.to_device(self.sim.cl_q, self.d1_